In [1]:
import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

In [2]:
import tweepy
import user_tweets
import hu2004sentiment as sa
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tweepy import TweepError

pd.options.display.max_columns = 50
pd.options.display.max_rows= 50
pd.options.display.width= 120

In [3]:
consumer_key = "Y9LSrr5Ctyco96MxuB5lVrXLE" # Use your own key. To get a key https://apps.twitter.com/
consumer_secret = "MZ0ZzvzcyDFQAri4YFQ1NUJIUdyaaKr22YoHt1sdE6yVkUPOk4"

access_token = "1718965579-EbDVvYloIQWEMdrC62fjSDHnkmjW3a876S33G7h"
access_token_secret = "fxxW6evy2Gzlxc0k16d4Lckpkd2QQXW2obXFGZh5divHe"

auth = tweepy.OAuthHandler(consumer_key=consumer_key, consumer_secret=consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

## Extracting #ihateapple tweet IDs

1. Search the #ihateapple in twitter website ([https://search.twitter.com](https://search.twitter.com))
2. Put the following command into the js console
```javascript
    allTweets = $('li.stream-item');
    tweetIds = ''; 
    for(let i=0 ; i<allTweets.length ; i++) 
        tweetIds += allTweets[i].getAttribute('data-item-id') + '\n';
```
3. Save the log

In [4]:
extraction_needed = False

if extraction_needed:
    extracted_tweets = []
    extracted_tweets_ids = set()
    with open('../../resources/dataset/IHateApple-tweetIds.log') as f:

        tweetIds = f.readlines() #.split('\n')
        for tweetId in tweetIds:
            tweetId = tweetId.strip()
            try:
                tweet = api.get_status(tweetId)
                extracted_tweets.append(tweet)
                extracted_tweets_ids.add(tweetId)
                print "%4d"%len(extracted_tweets), tweetId, tweet.text[:40]
            except TweepError as e:
                print e


    import pickle

    print len(extracted_tweets)
    print len(extracted_tweets_ids)

    with open( "extracted_tweets.pickle", "wb" ) as extracted_tweets_f:
        pickle.dump(extracted_tweets, extracted_tweets_f)

    with open( "extracted_tweets_ids.pickle", "wb" ) as extracted_tweets_ids_f:
        pickle.dump(extracted_tweets_ids, extracted_tweets_ids_f)

In [5]:
import pickle

extracted_tweets = []
extracted_tweets_ids = set()

with open( "extracted_tweets.pickle", "rb" ) as extracted_tweets_f:
    extracted_tweets = pickle.load(extracted_tweets_f)

with open( "extracted_tweets_ids.pickle", "rb" ) as extracted_tweets_ids_f:
    extracted_tweets_ids = pickle.load(extracted_tweets_ids_f)
    
print len(extracted_tweets)
print len(extracted_tweets_ids)

1264
1264


In [6]:
# Load the related tweets excel
import xlrd
import openpyxl
from pandas import ExcelFile

In [7]:
related_tweets_pd = pd.read_excel("related_tweets.xlsx")
user_rel_tweets_map = {}
for idx, row in related_tweets_pd.iterrows():
    user = row.user.strip()
    text = row.text 
    # print type(text) != type(unicode()), text
    if type(text) != type(unicode()):
        text = unicode(text).strip()
        
    text = text.encode('ascii', 'ignore').strip()
    # print text[:40]
    if text != u'null' and text != u'nan' and len(text) != 0:
        user_rel_tweets_map[user] = text.strip()
    else:
        print user,'\t', text

Myline_Shreya 	nan
NDN_DCaldwell 	nan
16thman 	nan
___COLONEL___ 	nan
BoyNextRawr 	nan
stefanie__92 	nan
The140plus 	nan
... 	null
... 	null
... 	null
MadeIn_Asiaa 	nan
JRadhaakr 	nan
inndiaaa_ 	nan
xcourterrsx 	nan
CaraHuyck 	nan
teeyaaraa_ 	nan
starkavaauthor 	nan
_cytch 	nan
GinaGiles7 	nan
_JessieJaye 	nan
ANGELAFAITHD 	nan
EmilyOwens14 	nan
RealSlimSimon 	nan
Darrellwa3 	nan
DPrice_IsRight 	nan
... 	null
GrandeKxJ 	nan
... 	null
... 	null
__contourqueen 	nan
halihall 	nan
IveG0tFangz 	nan
del_43 	nan
... 	null
RealSlimSimon 	nan
RealSlimSimon 	nan


In [8]:
def process_tweets(tweets):
    id_list = [tweet.id for tweet in tweets]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data
    data_set["text"] = [tweet.text for tweet in tweets]
    data_set["created_at"] = [tweet.created_at for tweet in tweets]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in tweets]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in tweets]
    data_set["is_replay_to"] = [tweet.in_reply_to_status_id is not None for tweet in tweets]
    data_set["source"] = [tweet.source for tweet in tweets]
    
    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in tweets]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in tweets]
    data_set["user_name"] = [tweet.author.name for tweet in tweets]
    data_set["user_created_at"] = [tweet.author.created_at for tweet in tweets]
    data_set["user_description"] = [tweet.author.description for tweet in tweets]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in tweets]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in tweets]
    data_set["favourites_count"] = [tweet.author.favourites_count for tweet in tweets]
    data_set["statuses_count"] = [tweet.author.statuses_count for tweet in tweets]
    data_set["user_location"] = [tweet.author.location for tweet in tweets]
    
    with open('related_tweets.csv', 'w') as rt_csv:
        related_tweets = []
        for tweet in tweets:
            if tweet.author.screen_name not in user_rel_tweets_map:
                while True:
                    user_rel_tweets = 'null'
                    break
                    
                    try:
                        user_rel_tweets = user_tweets.get_related_tweets_for_user(tweet.author.screen_name,'apple')
                        user_rel_tweets = user_rel_tweets.replace('\n', ' ')
                        user_rel_tweets = user_rel_tweets.replace(',', ' ')
                        user_rel_tweets = user_rel_tweets.replace('"', ' ')
                        user_rel_tweets = user_rel_tweets.replace("'", ' ')
                        
                        if type(user_rel_tweets) != type(unicode()):
                            user_rel_tweets = unicode(user_rel_tweets).strip()
                        # user_rel_tweets = user_rel_tweets.encode('ascii', 'ignore').strip()
                        
                        user_rel_tweets_map[tweet.author.screen_name] = user_rel_tweets.strip()
                        
                        rt_csv.write(str(len(related_tweets)) + "," + tweet.author.screen_name + ',"' + user_rel_tweets + '"\n')
                        rt_csv.flush()
                        
                        break

                    except tweepy.error.TweepError as e:
                        print 'retrying', str(e)[:70]
                try:
                    print str(len(related_tweets)) + ", " + tweet.author.screen_name + ", " + user_rel_tweets[:40]
                except UnicodeDecodeError as e:
                    print str(len(related_tweets)) + ", " + tweet.author.screen_name + ', ' + str(e)[:40]
            else :
                user_rel_tweets = user_rel_tweets_map[tweet.author.screen_name]
                
                rt_csv.write(str(len(related_tweets)) + "," + tweet.author.screen_name + ',"' + user_rel_tweets + '"\n')
                rt_csv.flush()
                
                print str(len(related_tweets)) + ", " + tweet.author.screen_name + user_rel_tweets[:40]
            
                
            
            related_tweets.append(user_rel_tweets)
            
        data_set["related_tweets"] = related_tweets

    return data_set

data_set = process_tweets(extracted_tweets)

0, Lizzyhalstedrt @aclu: sessions seems to think  the h
1, emshelx@bamblingsofnafy yeh done them all x i v
2, t_herbo2if you text me im not ignoring you  my p
3, 84daniellemariewhy has my phone shut off on its own 3 t
4, kaitlynsonkesomeone send me an apple music link to t
5, dknaubbso frustrated with technology. #ihateapp
6, Soff_fernandezbdang where is my nazi flag emoji whereee
7, TrivolutionGameour third and final show of the night at
8, Hinderz30regret selling my ipod to fund my trip. 
9, 352Andrew@hesgoingtoscore fried apple pie festiva
10, TGJones_62rt @melissaguenette: an apple a day can 
11, baileybooo1996i want that new @google phone pixel bc #
12, baileybooo1996i want that new @google phone pixel bc #
13, concepcionem_my messages are going through a struggle
14, DWL_SLC@apple is the chastity belt of technolog
15, grifflaar@adam_askey fuckin hell i said i was out
16, wendyjensenmus#itunes  you have pissed me off one too 
17, ItsHeather7what s so good about apple? why do you w

In [11]:
data_set.head(1)

,id,text,created_at,retweet_count,favorite_count,is_replay_to,source,user_id,user_screen_name,user_name,user_created_at,user_description,user_followers_count,user_friends_count,favourites_count,statuses_count,user_location,related_tweets
0,813780398836682752,switched back to iphone this weekend but WTF @...,2016-12-27 16:15:41,0,1,False,Mobile Web (M5),744927675983204352,Lizzyhalsted,Lizzyhalsted,2016-06-20 16:19:32,"Attorney, writer & mom of #Konjo #Habesha \nRa...",140,337,3166,5126,"Chicago, IL",rt @aclu: sessions seems to think the high-pr...


In [12]:
import openpyxl
from pandas import ExcelWriter

writer = ExcelWriter('../../resources/dataset/tweets.xlsx')
data_set.to_excel(writer,'Sheet1')
writer.save()